In [2]:
from datetime import datetime, timedelta, date
from typing import Annotated, Union

from fastapi import Depends, FastAPI, HTTPException, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from passlib.context import CryptContext
from pydantic import BaseModel, EmailStr
from enum import Enum
import psycopg2
import pandas as pd

In [3]:
DB_CON = {
        "host"      : 'rds-ticket-app-prod.czgosi3nm3l7.eu-central-1.rds.amazonaws.com',
        "database"  : 'public',
        "user"      : 'ticketappuser',
        "password"  : 'GA8yDbApaTthvqgH16V0jg==',
        'port'      : '5432'
    }

def format_value_for_sql(value):
    if value is None:
        return "NULL"
    elif isinstance(value, int):
        return str(value)
    elif isinstance(value, float):
        return str(value)
    elif isinstance(value, bool):
        return str(int(value))  # Convert bool to 0 or 1
    else:
        # If the data type is not recognized, raise an error or handle it accordingly
        return f"""'{str(value).replace("'", "''")}'"""

def read_data(sql):
    conn = psycopg2.connect(**DB_CON)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    cols = [str(col[0]) for col in cur.description ]
    conn.close()
    return pd.DataFrame(rows, columns=cols)

In [4]:
class Role(str, Enum):
    admin = "admin"
    user = "user"

class User(BaseModel):
    username: str
    email: EmailStr
    role: Role
    name: Union[str, None] = None
    disabled: Union[bool, None] = None

In [5]:
role = Role('admin')
user = User(username='med', email='med@rebike.com', role=role, name='medd', disabled=False)
user

User(username='med', email='med@rebike.com', role=<Role.admin: 'admin'>, name='medd', disabled=False)

In [ ]:
import psycopg2

try:
    # Establish a connection to the database
    conn = psycopg2.connect(database="your_database", user="your_user", password="your_password", host="your_host", port="your_port")

    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()

    # Your SQL INSERT query with RETURNING clause
    insert_query = "INSERT INTO your_table (column1, column2) VALUES (%s, %s) RETURNING column1, column2, other_columns;"

    # Specify the values for the new record
    values = ('value1', 'value2')

    # Execute the INSERT query
    cursor.execute(insert_query, values)

    # Fetch the newly inserted row
    new_row = cursor.fetchone()

    # Commit the changes to the database
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()

    # Now, new_row contains the values of the columns specified in the RETURNING clause
    print("Newly inserted row:", new_row)

except psycopg2.Error as e:
    # Handle the exception (print or log the error message, rollback changes, etc.)
    print("Error:", e)

finally:
    # Ensure that the connection is properly closed
    if conn is not None:
        conn.close()


In [14]:
obj = {
        "username": 'usdsder11',
        "email": 'user@email.com',
        "role": 'user',
        "name": 'new_user.name',
        "password": 'pwd',
        "disabled": False
        }
keys = ', '.join([f'"{o}"' for o in obj.keys()])
tags = ', '.join(['%s' for i in obj.values()])
sql = f"""
Insert into users ({ keys }) values ({ tags })
returning *
"""
sql

'\nInsert into users ("username", "email", "role", "name", "password", "disabled") values (%s, %s, %s, %s, %s, %s)\nreturning *\n'

In [15]:
conn = psycopg2.connect(**DB_CON)
cur = conn.cursor()
cur.execute(sql, list(obj.values()))
conn.commit()

{'id': 15,
 'username': 'usdsder11',
 'email': 'user@email.com',
 'password': 'pwd',
 'role': 'user',
 'name': 'new_user.name',
 'disabled': False}